In [ ]:
import torch

print(f"파이토치 버전: {torch.__version__}")
print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
print(f"CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.version}")
print(f"bf16 지원 여부: {torch.cuda.is_bf16_supported()}")
print(f"현재 GPU : {torch.cuda.current_device()}")

In [ ]:
import torch

def check_vram_torch():
    # 1. 현재 텐서들이 실제로 점유하고 있는 메모리
    allocated = torch.cuda.memory_allocated() / (1024**3)
    # 2. PyTorch가 OS로부터 할당받아 캐싱하고 있는 전체 메모리
    reserved = torch.cuda.memory_reserved() / (1024**3)
    # 3. 해당 GPU의 전체 용량
    total = torch.cuda.get_device_properties(0).total_memory / (1024**3)

    print(f"--- GPU Memory Status ---")
    print(f"Allocated: {allocated:.2f} GB")
    print(f"Reserved:  {reserved:.2f} GB")
    print(f"Total:     {total:.2f} GB")
    print(f"Free (in Reserved): {reserved - allocated:.2f} GB")
    print(f"--------------------------")

check_vram_torch()

In [ ]:
from huggingface_hub import snapshot_download

save_path = "/kaggle/working/Exaone-3.5-2.4B-Instruct-AWQ"

snapshot_download(
    repo_id="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct-AWQ",
    local_dir=save_path,
    local_dir_use_symlinks=False,  # 실제 파일을 해당 경로에 직접 복사/다운로드
    max_workers=8 # 병렬 다운로드 속도 향상
)

In [ ]:
from datasets import Dataset
import pandas as pd
import glob

file_path = "/kaggle/input/datasets/jungseong2/goyanglibrary-faq1/FAQ1.xlsx"
df = pd.read_excel(file_path)
dataset = Dataset.from_pandas(df)

print(dataset)
print(dataset[0])

In [ ]:
import json
import os
import asyncio, aiofiles
import wandb
from tqdm.asyncio import tqdm as async_tqdm
from openai import AsyncOpenAI
import weave
from kaggle_secrets import UserSecretsClient

if wandb.run is not None:
    wandb.finish()

try:
    user_secrets = UserSecretsClient()
    wandb_key = user_secrets.get_secret("wandb")
    os.environ["WANDB_API_KEY"] = wandb_key
    wandb.login(key=wandb_key, relogin=True)
except Exception as e:
    print(f"Could not retrieve W&B secret: {e}")

PROJECT_NAME="library-faq-augmentation-AWQ"

config = {
    "model": "/kaggle/working/Exaone-3.5-2.4B-Instruct-AWQ",
    "quantization": "awq",
    "engine": "vLLM-v1",
    "temperature": 0.7,
    "max_tokens" : 2048,
    "top_p" : 0.7,
    "max_model_len": 16384,
    "gpu_util": 0.7
}

wandb.init(project=PROJECT_NAME, name="EXAONE-3.5-2.4B-AWQ-vLLM-Run")
weave.init(PROJECT_NAME)
sample_table = wandb.Table(columns=["row_id", "question", "answer", "label"])
client = AsyncOpenAI(base_url="http://localhost:8000/v1", api_key="EMPTY")

In [ ]:
import subprocess
import time

# vLLM 서버를 백그라운드에서 실행
vllm_cmd = [
    "python3", "-m", "vllm.entrypoints.openai.api_server",
    "--model", "/kaggle/working/Exaone-3.5-2.4B-Instruct-AWQ",
    "--port", "8000",
    "--tensor-parallel-size", "2",
    "--gpu-memory-utilization", "0.8",
    "--max-model-len", "16384",
    "--quantization", "awq",
    "--trust-remote-code",
    "--distributed-executor-backend", "mp"
]

# 로그를 파일로 남기며 백그라운드 실행
with open("/kaggle/working/vllm_server.log", "w") as f:
    process = subprocess.Popen(vllm_cmd, stdout=f, stderr=f)

In [ ]:
!tail -n 40 /kaggle/working/vllm_server.log

In [ ]:
@weave.op()
async def generate_augmented_data_async(faq_content, row_id, config) :
    system_message = (
    "당신은 도서관 FAQ 데이터를 바탕으로 자연스러운 대화형 학습 데이터를 생성하는 전문가입니다.\n"
    "제공된 정보를 분석하여 다음 4가지 유형의 데이터를 생성하세요. 각 유형별로 최소 3개씩, 총 12개의 예시를 만드세요.\n\n"
    "이때 판단이 쉬운 예시와 보통, 어려운 예시 총 3가지 상황을 가정하세요."
    
    "### [답변 가이드라인]\n"
    "1. **label: 'yes' (긍정 확인)**\n"
    "   - 질문이 '~할 수 있나요?', '~인가요?'처럼 '가부'를 물을 때만 사용하세요.\n"
    "   - 문구: '네, 가능합니다! 도서관 규정에 따르면...'\n"
    "   - 활용: 질문의 조건이 FAQ와 일치할 때 사용.\n"
    "2. **label: 'no' (부정/제한)**\n"
    "   - 질문이 '~할 수 있나요?', '~인가요?'처럼 '가부'를 물을 때만 사용하세요.\n"
    "   - 문구: '죄송하지만 어렵습니다. 그 이유는...'\n"
    "   - 활용: 질문의 조건이 FAQ 규정에 어긋날 때 사용. '사실과 다르다'는 표현 대신 '규정상 어렵다'를 사용하세요.\n"
    "3. **label: 'info' (단순 정보 제공)**\n"
    "   - 질문이 '~은 무엇인가요?', '~은 어떻게 하나요?', '~은 언제인가요?'처럼 절차/시간/방법을 물을 때 사용하세요.\n"
    "   - 문구: '문의하신 내용에 대해 안내해 드리겠습니다. 관련 서류는...'\n"
    "   - 활용: 네/아니오 판단이 아닌 정보 질문에 사용.\n"
    "4. **label: 'false' (판단 불가)**\n"
    "   - 문구: '죄송합니다. 현재 제공된 정보만으로는 해당 내용을 확인하기 어렵습니다.'\n"
    "   - 활용: FAQ에 없는 내용이거나 질문이 모호할 때 사용.\n\n"
    
    "모든 답변은 실제 도서관 사서가 방문객에게 설명하듯 친절하고 자연스러운 문장으로 작성하세요."
    "판단 불가의 경우 문구 외에 다른 문장은 생성하지 마세요."
    "JSON 답변 외에 다른 문장은 일체 생성하지 마세요."
    )

    # 사용자가 정의한 user_content
    user_content = f"""
        [도서관 FAQ 정보]
        {faq_content}
        
        위 정보를 바탕으로 질문과 답변이 논리적으로 완벽하게 이어지는 JSON 데이터를 생성하세요.
        
        [출력 예시]:
        [
          {{"question": "임산부인데 남편이 대신 발급받을 수 있나요?", "answer": "네, 가능합니다! 도서관 규정에 따르면 임산부의 경우 대리 발급 대상에 포함됩니다.", "label": "yes"}},
          {{"question": "성인 직장인인데 친구가 대신 가도 되나요?", "answer": "죄송하지만, 해당 조건으로는 대리 발급이 어렵습니다. 대리 발급은 아동, 어르신, 장애인, 임산부로 대상이 제한되어 있기 때문입니다.", "label": "no"}},
          {{"question": "장애인 대리 발급 시 어떤 서류가 필요한가요?", "answer": "문의하신 내용에 대해 안내해 드리겠습니다. 장애인 복지카드 또는 장애인 증명서를 지참하시면 됩니다.", "label": "info"}},
          {{"question": "죄송합니다. 현재 제공된 정보만으로는 해당 내용을 확인하기 어렵습니다.", "label": "false"}}
        ]
    """
    
    try :
        chat_completion = await client.chat.completions.create(
            model=config["model"],  # MODEL_NAME 대신 config["model"] 사용
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_content},
            ],
            temperature=config["temperature"],
            max_tokens=config["max_tokens"],
            top_p=config["top_p"],
        )

        raw_output = chat_completion.choices[0].message.content

        if "```json" in raw_output:
            raw_output = raw_output.split("```json")[1].split("```")[0].strip()
        elif "```" in raw_output:
            raw_output = raw_output.split("```")[1].split("```")[0].strip()

        parsed_list = json.loads(raw_output)
        return parsed_list

    except Exception as e :
        print(f"Error at {row_id} : {e}")
        return []
    
async def process_dataset(dataset, config) :
    file_name = config["model"].split("/")[-1]
    output_file = f"library_faq_quant_{file_name}_vLLM_async.jsonl"
    async with aiofiles.open(output_file, "w", encoding="utf-8") as f:
        semaphore = asyncio.Semaphore(16)
        lock = asyncio.Lock()

        async def semaphore_task(row, idx) :
            async with semaphore :
                results = await generate_augmented_data_async(row["DES"], idx, config)
                
                if results:  # 결과가 있을 때만 쓰기
                    async with lock:

                        for item in results[:2]:
                            sample_table.add_data(idx, item.get("question"), item.get("answer"), item.get("label"))

                        for item in results:
                            item.update({
                                'faq': row['FAQ'], 
                                'title': row['TITLE'], 
                                'DES': row['DES']
                            })
                            await f.write(json.dumps(item, ensure_ascii=False) + "\n")
                
                return len(results)
            return 0

        tasks = [semaphore_task(row, i) for i, row in enumerate(dataset)]

        total_created = 0

        for coro in async_tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="데이터 생성"):
            total_created += await coro

    wandb.log({"augmented_samples": sample_table})
    print(f"✅ 생성 완료! 총 {total_created}개의 데이터가 저장되었습니다.")

    return output_file

In [ ]:
outputfile_name = await process_dataset(dataset, config)
artifact = wandb.Artifact(f"{config['model']}_{wandb.run.id}", type='dataset')
artifact.add_file(outputfile_name)
wandb.log_artifact(artifact)
wandb.finish()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json

data = []
with open("/kaggle/input/augmented-llibrary-faq-quant-exaone-jsonl/augmented_library_faq_quant_EXAONE_32B_vLLM.jsonl", "r", encoding="utf-8") as f :
    for line in f :
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [ ]:
print(df["label"].value_counts())

In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.info())
df = df.drop(columns="url")

In [ ]:
print(df.info())

In [ ]:
idx = df[df["answer"].isna()].index
df.drop(idx, inplace=True)

In [ ]:
df = df.reset_index(drop=True)
print(df.info())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json

# 2. 라벨 분포 확인 (균형성)
print("--- [1. Label Distribution] ---")
print(df['label'].value_counts())

plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='label', order=['yes', 'no', 'info', 'false'])
plt.title("Label Distribution")
# plt.savefig("/kaggle/working/label_distribution.png")
plt.savefig("/kaggle/working/label_distribution4.png")

In [ ]:
pivot = df.groupby(["title", "label"]).size().unstack(fill_value=0) # 인덱스 값을 컬럼으로
need_aug_idx = pivot[(pivot["yes"]<=1) | (pivot["no"]<=2) | (pivot["false"] <=2)]

print(pivot)
print(len(need_aug_idx))

In [ ]:
def get_target_labels(title, pivot_table):
    """특정 title에서 증강이 필요한 라벨 목록을 반환"""
    row = pivot_table.loc[title]
    targets = []
    # 부족한 라벨 판단 기준 (예: 3개 이하)
    if row.get("yes", 0) <= 3: targets.append("yes")
    if row.get("no", 0) <= 3: targets.append("no")
    if row.get("false", 0) <= 3: targets.append("false")
    return targets

In [ ]:
@weave.op()
async def generate_targeted_data_async(faq_content, target_labels, config):
    # 필요한 라벨에 대한 가이드라인 / 예시만 프롬프트로 넣어서, 다른 예시가 생성되지 않도록 하기
    all_guidelines = {
        "yes" : "질문이 '~할 수 있나요?', '~인가요?'처럼 '가부'를 물을 때만 사용하세요.\n - 문구: '네, 가능합니다! 도서관 규정에 따르면...'\n - 활용: 질문의 조건이 FAQ와 일치할 때 사용.\n",
        "no" : "질문이 '~할 수 있나요?', '~인가요?'처럼 '가부'를 물을 때만 사용하세요.\n - 문구: '죄송하지만 어렵습니다. 그 이유는...'\n - 활용: 질문의 조건이 FAQ 규정에 어긋날 때 사용. '사실과 다르다'는 표현 대신 '규정상 어렵다'를 사용하세요.\n",
        "info" : "질문이 '~은 무엇인가요?', '~은 어떻게 하나요?', '~은 언제인가요?'처럼 절차/시간/방법을 물을 때 사용하세요.\n 문구: '문의하신 내용에 대해 안내해 드리겠습니다. 관련 서류는...'\n - 활용: 네/아니오 판단이 아닌 정보 질문에 사용.\n",
        "false" : "문구: '죄송합니다. 현재 제공된 정보만으로는 해당 내용을 확인하기 어렵습니다.'\n - 활용: FAQ에 없는 내용이거나 질문이 모호할 때 사용.\n\n"
    }
    selected_guidelines = "\n".join([all_guidelines[l] for l in target_labels if l in all_guidelines])

    all_examples = {
        "yes": '{"question": "임산부인데 남편이 대신 발급받을 수 있나요?", "answer": "네, 가능합니다! 도서관 규정에 따르면 임산부의 경우 대리 발급 대상에 포함됩니다.", "label": "yes"}',
        "no": '{"question": "성인 직장인인데 친구가 대신 가도 되나요?", "answer": "죄송하지만, 해당 조건으로는 대리 발급이 어렵습니다. 대리 발급은 아동, 어르신, 장애인, 임산부로 대상이 제한되어 있기 때문입니다.", "label": "no"}',
        "info": '{"question": "장애인 대리 발급 시 어떤 서류가 필요한가요?", "answer": "문의하신 내용에 대해 안내해 드리겠습니다. 장애인 복지카드 또는 장애인 증명서를 지참하시면 됩니다.", "label": "info"}',
        "false": '{"question": "도서관 근처에 맛있는 식당이 어디인가요?", "answer": "죄송합니다. 현재 제공된 정보만으로는 해당 내용을 확인하기 어렵습니다.", "label": "false"}'
    }   

    target_str = ", ".join(target_labels)
    selected_examples = ",\n      ".join([all_examples[l] for l in target_labels if l in all_examples])
    system_message = (
        f"당신은 도서관 FAQ 데이터를 바탕으로 부족한 유형의 학습 데이터를 보충하는 전문가입니다.\n"
        f"제공된 정보를 분석하여 반드시 다음 라벨에 해당하는 데이터만 생성하세요 : **[{target_str}]**\n\n"
        f"이때 판단이 쉬운 예시와 보통, 어려운 예시 총 3가지 상황을 가정하여 각 상황별로 데이터를 생성하세요."
        f"각 라벨마다 최소 2~3개의 답변을 생성하세요"
        
        f"### [답변 가이드라인]\n{selected_guidelines}\n\n"
        "모든 답변은 실제 도서관 사서가 방문객에게 설명하듯 친절하고 자연스러운 문장으로 작성하세요."
        f"**[{target_str}]**에 해당하지 않는 라벨은 절대로 생성하지 마세요"
        "판단 불가의 경우 문구 외에 다른 문장은 생성하지 마세요."
        "JSON 답변 외에 다른 문장은 일체 생성하지 마세요."
    )

    # 사용자가 정의한 user_content
    user_content = f"""
    [도서관 FAQ 정보]
    {faq_content}
    
    위 정보를 바탕으로 질문과 답변이 논리적으로 완벽하게 이어지는 JSON 데이터를 생성하세요.
    
    [출력 예시]:
    {selected_examples}
    """
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content},
    ]

    print(messages)

    try :
        chat_completion = await client.chat.completions.create(
            model=config["model"],  # MODEL_NAME 대신 config["model"] 사용
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_content},
            ],
            temperature=config["temperature"],
            max_tokens=config["max_tokens"],
            top_p=config["top_p"],
        )

        raw_output = chat_completion.choices[0].message.content

        if "```json" in raw_output:
            raw_output = raw_output.split("```json")[1].split("```")[0].strip()
        elif "```" in raw_output:
            raw_output = raw_output.split("```")[1].split("```")[0].strip()

        parsed_list = json.loads(raw_output)
        return parsed_list

    except Exception as e :
        print(f"Error : {e}")
        return []

In [ ]:
import shutil

async def process_targeted_dataset(dataset, pivot_table, config) :
    file_name = config["model"].split("/")[-1]

    # original_input_file = "/kaggle/input/augmented-llibrary-faq-quant-exaone-jsonl/augmented_library_faq_quant_EXAONE_32B_vLLM.jsonl"
    # original_input_file = f"/kaggle/working/augmented_library_faq_quant_{file_name}_v2.jsonl"
    original_input_file = f"/kaggle/working/augmented_library_faq_quant_{file_name}_v3.jsonl"
    output_file = f"/kaggle/working/augmented_library_faq_quant_{file_name}_v4.jsonl"

    shutil.copy(original_input_file, output_file)
    
    async with aiofiles.open(output_file, "a", encoding="utf-8") as f:
        semaphore = asyncio.Semaphore(16)
        lock = asyncio.Lock()

        async def semaphore_task(row, idx) :
            target_labels = get_target_labels(row["TITLE"], pivot_table)
            print(target_labels)
            
            async with semaphore :
                results = await generate_targeted_data_async(row["DES"], target_labels, config)
                
                if results:  # 결과가 있을 때만 쓰기
                    async with lock:
                        for item in results[:2]:
                            sample_table.add_data(idx, item.get("question"), item.get("answer"), item.get("label"))

                        for item in results:
                            item.update({
                                'faq': row['FAQ'], 
                                'title': row['TITLE'], 
                                'DES': row['DES']
                            })
                            await f.write(json.dumps(item, ensure_ascii=False) + "\n")
                
                return len(results)
            return 0

        tasks = [semaphore_task(row, i) for i, row in enumerate(dataset)]

        total_created = 0

        for coro in async_tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="데이터 생성"):
            total_created += await coro

    wandb.log({"augmented_samples": sample_table})
    print(f"✅ 생성 완료! 총 {total_created}개의 데이터가 저장되었습니다.")

    return output_file

In [ ]:
df_dataset = dataset.to_pandas()

target_titles = need_aug_idx.index.tolist()
filtered_df = df_dataset[df_dataset['TITLE'].isin(target_titles)][['FAQ', 'TITLE', 'DES']].drop_duplicates()

target_data_list = filtered_df.to_dict('records')
print(len(target_data_list))

In [ ]:
outputfile_name = await process_targeted_dataset(target_data_list, pivot, config)
artifact = wandb.Artifact(f"{config['model']}_{wandb.run.id}", type='dataset')
artifact.add_file(outputfile_name)
wandb.log_artifact(artifact)
wandb.finish()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json

data = []
with open("/kaggle/working/QA-FineTune/도서관_QA_FineTune/augmented_library_faq_quant_Exaone-3.5-7.8B-Instruct-AWQ_v4.jsonl", "r", encoding="utf-8") as f :
    for line in f :
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [ ]:
print(df["label"].value_counts())

In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.info())
df = df.drop(columns="url")

In [ ]:
print(df.info())

In [ ]:
idx = df[df["answer"].isna()].index
df.drop(idx, inplace=True)

In [ ]:
df = df.reset_index(drop=True)
print(df.info())

In [ ]:
from datasets import Dataset

df['faq'] = df['faq'].astype(str)
hf_dataset = Dataset.from_pandas(df[["question", "answer","DES"]])
seed = 42

ds_split = hf_dataset.train_test_split(test_size=0.2, seed=seed)
train_val_split = ds_split["train"].train_test_split(test_size = 0.3, seed=seed)

from datasets import DatasetDict

final_dataset = DatasetDict({
    'train': train_val_split['train'],
    'val': train_val_split['test'],
    'test': ds_split['test']
})

train_dataset, val_dataset, test_dataset = final_dataset['train'], final_dataset['val'], final_dataset['test']

In [ ]:
print(train_dataset)
print(train_dataset[0])

In [ ]:
os.environ["HF_TOKEN"]=hf_token

In [ ]:
# 생성한 데이터 셋 저장
from datasets import Dataset, DatasetDict
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

hf_token = user_secrets.get_secret("hf_token")

login(token=hf_token)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

dataset_dict.push_to_hub("JungSeong2/library-qa")

In [ ]:
import os
import shutil
from kaggle_secrets import UserSecretsClient

# 1. 환경 설정
user_secrets = UserSecretsClient()
GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN") 
USER_EMAIL = user_secrets.get_secret("USER_EMAIL")
USER_NAME = "JungSeong"
REPO_NAME = "QA-FineTune"
SUB_DIR = "도서관_QA_FineTune"
FILE_NAME = "label_distribution4.png"

# 2. Git 설정
!git config --global user.email "{USER_EMAIL}"
!git config --global user.name "{USER_NAME}"

# 3. 레포지토리 클론 (폴더가 없을 때만)
if not os.path.exists(REPO_NAME):
    !git clone https://{GITHUB_TOKEN}@github.com/{USER_NAME}/{REPO_NAME}.git
else:
    print("✅ Repository already exists.")

# 4. 폴더 생성 및 파일 복사 (Python 라이브러리 활용)
working_path = "/kaggle/working"
repo_path = os.path.join(working_path, REPO_NAME)
target_dir = os.path.join(repo_path, SUB_DIR)

# 만약 같은 이름의 '파일'이 존재하면 삭제하고 폴더를 만듭니다.
if os.path.exists(target_dir) and not os.path.isdir(target_dir):
    os.remove(target_dir)

os.makedirs(target_dir, exist_ok=True)

# 파일 복사
source_file = os.path.join(working_path, FILE_NAME)
destination_file = os.path.join(target_dir, FILE_NAME)

if os.path.exists(source_file):
    shutil.copy(source_file, destination_file)
    print(f"✅ 파일 복사 완료: {destination_file}")
else:
    print(f"❌ 원본 파일을 찾을 수 없습니다: {source_file}")

# 5. Commit & Push
%cd {repo_path}
!git add .
!git commit -m "Update: {SUB_DIR}/{FILE_NAME}"
!git push origin main

print(f"\n🚀 모든 작업이 완료되었습니다!")

In [ ]:
# 결측치 처리 및 데이터 증강 이후 파인튜닝 (LoRA + DeepSpeed)

In [ ]:
# 3. 학습 시킬 모델 불러오기

from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig, 
    TrainingArguments,
)
from datasets import Dataset
import os, torch, json, wandb, subprocess
from sklearn.model_selection import train_test_split
import torch.nn as nn
from peft import (
    get_peft_model,
    LoraConfig, 
    TaskType,
    prepare_model_for_kbit_training
)

In [ ]:
from huggingface_hub import snapshot_download

save_path = "/kaggle/working/Exaone-3.5-2.4B-Instruct"

snapshot_download(
    repo_id="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    local_dir=save_path,
    local_dir_use_symlinks=False,  # 실제 파일을 해당 경로에 직접 복사/다운로드
    max_workers=8 # 병렬 다운로드 속도 향상
)

### DeepSpeed 라이브러리
https://huggingface.co/docs/transformers/ko/deepspeed?zero-config=ZeRO-2

In [ ]:
# DeepSpeed config 생성
import json

ds_config = {
    "fp16": { "enabled": True }, # 16비트 부동소수점 연산 (T4 GPU 최적화)
    "zero_optimization": {
        "stage": 2, # 옵티마이저와 그래디언트를 분산 저장
        "allgather_partitions": True, # 분산 처리된 GPU 파라미터 모을 때 더 효율적인 알고리즘 사용
        "allgather_bucket_size": 2e8, # 통신할 때 데이터를 묶어서 보내는 바구니 크기
        "overlap_comm": True, # 연산과 통신을 동시에 진행, 병목을 줄임
        "reduce_scatter": True, # 통신 오버헤드 감소
        "reduce_bucket_size": 2e8, # 그래디언트 감소 연산 시 데이터를 묶는 크기
        "contiguous_gradients": True # 그래디언트를 메모리상에 연속적으로 배치
    },
    "gradient_accumulation_steps": "auto", # TrainingArguments에 설정한 gradient_accumulation_steps 값
    "gradient_clipping": "auto", # 그래디언트 폭주를 막기 위한 클리핑(Max Norm) 값을 자동으로 가져옵니다
    "steps_per_print": 10, # DeepSpeed 내부의 상태(처리량, 메모리 등)를 얼마나 자주 로그에 찍을지
    "train_batch_size": "auto", # 
    "train_micro_batch_size_per_gpu": "auto", # GPU 한 대당 한 번에 처리하는 배치 사이즈
    "wall_clock_breakdown": False
}

with open("/kaggle/working/ds_config.json", "w") as f:
    json.dump(ds_config, f)

In [ ]:
with open("/kaggle/working/ds_config.json", "r") as f:
    print(f.read())

In [ ]:
import json
import os
import wandb
from datetime import datetime

if wandb.run is not None:
    wandb.finish()

try:
    user_secrets = UserSecretsClient()
    wandb_key = user_secrets.get_secret("wandb")
    os.environ["WANDB_API_KEY"] = wandb_key
    wandb.login(key=wandb_key, relogin=True)
except Exception as e:
    print(f"Could not retrieve W&B secret: {e}")

PROJECT_NAME="library-faq-finetune"

os.environ["WANDB_PROJECT"] = PROJECT_NAME # 프로젝트 이름
os.environ["WANDB_RUN_ID"] = "library_QA_v1" # 노트북 고유 ID
os.environ["WANDB_RESUME"] = "allow" # 해당 노트북에서 학습을 이어서 진행할 것인지

wandb.init(
    project=os.environ["WANDB_PROJECT"],
    id=os.environ["WANDB_RUN_ID"],
    resume=os.environ["WANDB_RESUME"],
    name=datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 대시보드에 표시될 이름
)

DataCollatorForCompletionOnlyLM 안되는 현상 발생 </br>
!deepspeed --num_gpus=2 QA-FineTune/도서관_QA_FineTune/train_with_deepspeed.py

In [ ]:
%%writefile /kaggle/working/QA-FineTune/도서관_QA_FineTune/train_with_deepspeed.py
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.integrations import HfDeepSpeedConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login

MODEL_ID = "/kaggle/working/Exaone-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def generate_prompts(example):  # 단일 예시 처리
    system_message = (
        "당신은 도서관 운영에 대한 전문적인 지식을 가진 인공지능 사서입니다."
        "제공된 [도서관 정보]를 바탕으로 사용자의 질문에 정확하고 친절하게 답하십시오. "
        "정보에 없는 내용은 함부로 추측하지 말고 정중히 확인이 어렵다고 답하세요."
    )
    
    question = example["question"]
    answer = example["answer"]
    context = example["DES"]
    
    user_content = (
        f"### [도서관 정보]\n{context}\n\n"
        f"### [질문]\n{question}\n\n"
        f"### [지시 사항]\n"
        f"1. 친절한 말투로 규정에 근거하여 답변할 것.\n"
        f"2. 3문단 이내로 답변할 것.\n"
        f"3. 답변 끝에 지시 사항을 반복하지 말 것."
    )
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": answer}
    ]
    
    full_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    return full_prompt

def train_loop(dataset):
    local_rank = int(os.environ.get("LOCAL_RANK", 0))
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29501'
    
    # DeepSpeed 수동 초기화 (NoneType 에러 방지)
    dschf = HfDeepSpeedConfig("/kaggle/working/ds_config.json")
    
    train_dataset = dataset["train"]
    val_dataset = dataset["validation"]
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        trust_remote_code=True
    )  # device_map 제거: DeepSpeed가 처리
    
    model.get_input_embeddings = lambda: model.transformer.wte
    
    modules = ["q_proj", "k_proj", "v_proj", "out_proj"]
    peft_config = LoraConfig(
        task_type="CAUSAL_LM",
        inference_mode=False,
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=modules
    )
    model = get_peft_model(model, peft_config)
    
    training_args = SFTConfig(
        output_dir="./SFT",
        num_train_epochs=1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        optim="paged_adamw_32bit",
        learning_rate=5e-5,
        bf16=False,
        fp16=True,
        max_seq_length=2048,
        logging_steps=10,
        completion_only_loss=False,
        packing=False,
        deepspeed="/kaggle/working/ds_config.json",
        report_to="none"
    )
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        args=training_args,
        formatting_func=generate_prompts,
        processing_class=tokenizer
    )
    
    print(f"Model device: {next(model.parameters()).device}")
    trainer.accelerator.print(trainer.model)
    
    trainer.train()
    
    final_save_path = "./SFT/final"
    os.makedirs(final_save_path, exist_ok=True)
    if trainer.is_world_process_zero():
        trainer.model.save_model(final_save_path)
        tokenizer.save_pretrained(final_save_path)

if __name__ == "__main__":
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
        print("Logged in to HuggingFace")
    dataset = load_dataset("JungSeong2/library-qa", token=hf_token)
    
    train_loop(dataset)

In [ ]:
from datasets import Dataset, DatasetDict
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

hf_token = user_secrets.get_secret("hf_token")

login(token=hf_token)

os.environ["HF_TOKEN"]=hf_token